# Print dataset in format for model

In [2]:
import pandas as pd
import numpy as np2
import math
import matplotlib.pyplot as plt
import datetime as dt
import time
import random
from refainedLibrary_v3 import *
from statsmodels.tsa.api import Holt
from scipy.interpolate import *

from IPython.display import clear_output

from tqdm.notebook import tqdm



trajectories_2_resampled_04s SINGLE SMOOTH

trajectories_2_resampled_04s_bis DOPPIO SMOOTH

In [3]:
rd = Refained_Data('sCREEN-dataset.txt')
#trajectories = rd.read_trajectories("trajectories_2_resampled_04s_bis.csv")
trajectories = rd.read_trajectories("trajectories_2_resampled_1s_shifted.csv")

In [4]:
len(trajectories)

4283

In [8]:
# DEPRECATED

"""
    Set the points in format for the models
    data: dataframe of points order by "time"
    time_delta: number of seconds for frame. Every frame is bigger as time_delta and inside it there are a certain number of point
    
    RETURN: dataframe with n collumn (n = # of frame) and 4 rows (n_frame, n_traj, x, y)
"""

def get_data_in_format(data, time_delta):
    return_df = pd.DataFrame(columns = ["n_frame", "n_traj", "x", "y"])
    end = st = 0
    n_frame = 0
    
    for i in tqdm(range(len(data))):
        delta_time = data.iloc[i]["time"] - data.iloc[st]["time"]
        
        if delta_time.seconds <= time_delta - 1:
            end = i
        else:
            time_frame = data.iloc[st:end + 1]                                  
            time_frame.drop(["id", "tag_id", "time"], axis = 1, inplace = True)
            time_frame["n_frame"] = n_frame
            return_df = return_df.append(time_frame)                    
            n_frame += 1            
            end = st = i
        
        if i == len(data)-1:
            time_frame = data.iloc[st:end + 1]                                  
            time_frame.drop(["id", "tag_id", "time"], axis = 1, inplace = True)
            time_frame["n_frame"] = n_frame
            return_df = return_df.append(time_frame)      

    return return_df.T


"""
    Given a list return the data ready for the model
    trajecotries: list of dataframe. every element inside the list is a trajectory
    thrs: treshold. How many traj for training and how many for test set
    
    RETURN: two dataframe. The first is the training set, the second is the test set
"""

def get_data_4_model(trajectories, thrs):
    #trajectories = rd.read_trajectories(PATH)
        
    resampled_traj = list(map(lambda x: x.iloc[::2], trajectories))
    all_trajectories = pd.concat(resampled_traj)
    
    max_n = 1000
    n_4_train = random.sample(range(0,max_n+1), int(max_n*thrs))
    n_4_test = [x for x in range(max_n) if x not in n_4_train]   

    train = all_trajectories[all_trajectories["n_traj"].isin(n_4_train)]
    train = train.sort_values('time').reset_index(drop=True)
    test = all_trajectories[all_trajectories["n_traj"].isin(n_4_test)]
    test = test.sort_values('time').reset_index(drop=True)
    
    train_data = get_data_in_format(train, 2)
    test_data = get_data_in_format(test, 2)
    
    print(n_4_train)
    print(n_4_test)
    
    return train_data, test_data


In [ ]:
n_4_train = [27, 375, 855, 418, 494, 155, 387, 72, 761, 322, 632, 105, 707, 216, 189, 837, 714, 637, 720, 102, 763, 486, 265, 575, 944, 470, 610, 33, 223, 71, 676, 294, 424, 843, 733, 315, 542, 678, 998, 159, 850, 164, 729, 454, 281, 19, 764, 895, 749, 514, 667, 563, 926, 701, 948, 280, 510, 721, 383, 565, 796, 726, 661, 854, 451, 672, 461, 662, 521, 849, 477, 4, 419, 108, 480, 668, 917, 256, 560, 857, 392, 278, 459, 644, 285, 879, 207, 196, 329, 231, 943, 533, 420, 253, 396, 446, 183, 856, 436, 653, 983, 212, 705, 484, 24, 753, 427, 463, 112, 364, 803, 145, 327, 469, 195, 977, 935, 807, 97, 267, 411, 633, 262, 15, 810, 896, 464, 673, 61, 670, 127, 111, 773, 509, 430, 621, 694, 890, 185, 912, 273, 408, 846, 601, 985, 723, 180, 40, 296, 945, 402, 94, 822, 762, 940, 14, 299, 30, 900, 959, 175, 326, 276, 790, 32, 76, 320, 608, 9, 248, 274, 738, 37, 176, 540, 232, 138, 899, 305, 150, 354, 656, 934, 6, 643, 567, 630, 113, 671, 252, 894, 902, 66, 871, 526, 980, 68, 674, 536, 853, 544, 18, 204, 357, 523, 747, 590, 336, 814, 445, 695, 126, 441, 901, 571, 47, 246, 539, 972, 892, 782, 239, 548, 214, 792, 473, 114, 46, 524, 642, 995, 646, 949, 429, 335, 860, 266, 905, 226, 946, 641, 168, 908, 817, 845, 190, 765, 74, 684, 868, 928, 130, 657, 566, 602, 652, 927, 920, 75, 541, 260, 677, 639, 768, 341, 755, 323, 468, 953, 457, 611, 829, 89, 893, 347, 623, 906, 440, 727, 936, 620, 472, 227, 597, 404, 505, 604, 629, 797, 852, 337, 706, 585, 415, 499, 914, 497, 498, 704, 546, 235, 358, 433, 194, 5, 99, 307, 691, 182, 350, 699, 49, 91, 425, 780, 960, 703, 53, 913, 618, 325, 709, 401, 651, 719, 388, 579, 140, 219, 83, 587, 181, 431, 982, 416, 941, 950, 339, 289, 17, 996, 742, 172, 434, 554, 766, 166, 545, 724, 872, 751, 57, 308, 527, 136, 121, 96, 148, 213, 417, 537, 7, 839, 774, 838, 690, 251, 103, 532, 784, 507, 199, 568, 979, 580, 177, 403, 154, 836, 400, 793, 600, 487, 328, 491, 835, 616, 832, 603, 974, 758, 504, 178, 798, 100, 581, 134, 162, 80, 708, 174, 125, 69, 988, 365, 543, 376, 907, 862, 135, 635, 60, 217, 767, 467, 962, 393, 976, 422, 475, 395, 410, 737, 617, 535, 772, 197, 660, 225, 479, 625, 654, 348, 378, 50, 508, 124, 55, 811, 233, 272, 489, 881, 330, 981, 363, 615, 277, 777, 123, 502, 110, 924, 391, 994, 98, 680, 750, 316, 84, 869, 512, 379, 390, 802, 367, 844, 258, 612, 880, 81, 528, 696, 744, 882, 592, 191, 506, 583, 78, 801, 370, 978, 687, 211, 675, 343, 156, 186, 117, 353, 986, 452, 993, 147, 870, 947, 466, 717, 242, 82, 143, 483, 556, 910, 728, 606, 897, 599, 318, 414, 638, 538, 964, 369, 342, 496, 312, 965, 718, 465, 284, 649, 760, 428, 748, 770, 922, 688, 929, 282, 800, 448, 250, 698, 815, 813, 255, 971, 295, 43, 366, 681, 501, 10, 818, 865, 552, 557, 735, 90, 79, 192, 362, 669, 968, 144, 1000, 722, 31, 785, 842, 786, 288, 884, 423, 659, 958, 825, 992, 0, 702, 222, 873, 519, 787, 831, 453, 2, 969, 209, 67, 520, 756, 118, 426, 820, 804, 990, 493, 173, 791, 133, 380, 371, 848, 874, 515, 101, 823, 20, 86, 56, 549, 298, 711, 52, 655, 444, 725, 598, 942, 42, 867, 45, 591, 740, 474, 270, 221, 700, 891, 54, 613, 627, 488, 476, 344, 503, 525, 741, 937, 293, 292, 28, 626, 824, 495, 485, 679, 769, 163, 407, 70, 462, 663, 202, 500, 781, 775, 241, 1, 346, 957, 732, 809, 841, 286, 398, 883, 712, 634, 664, 149, 961, 300, 551, 847, 249, 686, 119, 151, 275, 314, 373, 692, 93, 65, 840, 697, 577, 188, 628, 319, 447, 875, 951, 799, 171, 666, 915, 833, 331, 665, 349, 819, 861, 240, 588, 87, 877, 44, 932, 23, 334, 816, 582, 851, 650, 263, 522, 745, 471, 311, 129, 116, 904, 788, 257, 573, 304, 244, 137, 555, 16, 783, 531, 229, 886, 215, 157, 456, 594, 987, 8, 412, 106, 919, 866, 805, 310, 478, 210, 443, 435, 716, 570, 193, 450, 36, 455, 888, 386, 85, 95, 421, 889, 230, 834, 77, 658, 88, 41, 887, 967, 109, 806, 885, 283, 438, 161, 991, 460, 303, 911, 713, 921, 389, 22, 247, 306, 237, 62, 636, 736, 63, 384, 152, 930, 332, 333, 399, 576, 595, 682, 624, 437, 685, 997, 966]
n_4_test = [3, 11, 12, 13, 21, 25, 26, 29, 34, 35, 38, 39, 48, 51, 58, 59, 64, 73, 92, 104, 107, 115, 120, 122, 128, 131, 132, 139, 141, 142, 146, 153, 158, 160, 165, 167, 169, 170, 179, 184, 187, 198, 200, 201, 203, 205, 206, 208, 218, 220, 224, 228, 234, 236, 238, 243, 245, 254, 259, 261, 264, 268, 269, 271, 279, 287, 290, 291, 297, 301, 302, 309, 313, 317, 321, 324, 338, 340, 345, 351, 352, 355, 356, 359, 360, 361, 368, 372, 374, 377, 381, 382, 385, 394, 397, 405, 406, 409, 413, 432, 439, 442, 449, 458, 481, 482, 490, 492, 511, 513, 516, 517, 518, 529, 530, 534, 547, 550, 553, 558, 559, 561, 562, 564, 569, 572, 574, 578, 584, 586, 589, 593, 596, 605, 607, 609, 614, 619, 622, 631, 640, 645, 647, 648, 683, 689, 693, 710, 715, 730, 731, 734, 739, 743, 746, 752, 754, 757, 759, 771, 776, 778, 779, 789, 794, 795, 808, 812, 821, 826, 827, 828, 830, 858, 859, 863, 864, 876, 878, 898, 903, 909, 916, 918, 923, 925, 931, 933, 938, 939, 952, 954, 955, 956, 963, 970, 973, 975, 984, 989, 999]


In [5]:
"""
    Set the points in format for the models
    data: dataframe of points order by "time"
    time_delta: number of seconds for frame. Every frame is bigger as time_delta and inside it there are a certain number of point
    
    RETURN: dataframe with n collumn (n = # of frame) and 4 rows (n_frame, n_traj, x, y)
"""
def get_data_in_format(data, time_delta):
    return_df = pd.DataFrame(columns=["n_frame", "n_traj", "x", "y"])
    st = data.iloc[0]

    n_frame = 0

    dictionary_list = []

    dictionary_data = {
        "n_frame": n_frame,
        "n_traj": st["n_traj"],
        "x": st["x"],
        "y": st["y"]
    }
    dictionary_list.append(dictionary_data)


    for i in tqdm(range(1, len(data))):
        row = data.iloc[i]
        delta_time = row["time"] - st["time"]

        if delta_time.total_seconds() < time_delta:
            dictionary_data = {
                "n_frame": n_frame,
                "n_traj": row["n_traj"],
                "x": row["x"],
                "y": row["y"]
            }
            dictionary_list.append(dictionary_data)
        else:
            st = row.copy()
            n_frame += 1
            dictionary_data = {
                "n_frame": n_frame,
                "n_traj": row["n_traj"],
                "x": row["x"],
                "y": row["y"]
            }
            dictionary_list.append(dictionary_data)

    return_df = pd.DataFrame.from_dict(dictionary_list)
    return return_df.T

"""
    Given a list return the data ready for the model
    trajecotries: list of dataframe. every element inside the list is a trajectory
    thrs: treshold. How many traj for training and how many for test set
    
    RETURN: two dataframe. The first is the training set, the second is the test set
"""

def get_data_4_model(trajectories, thrs, dim_frame):

    all_trajectories = pd.concat(trajectories)
    dim = len(trajectories)
    
    
    n_4_train = random.sample(range(0,dim+1), int(dim*thrs))
    n_4_test = [x for x in range(dim) if x not in n_4_train]   

    
    train = all_trajectories[all_trajectories["n_traj"].isin(n_4_train)]
    train = train.sort_values('time').reset_index(drop=True)
    test = all_trajectories[all_trajectories["n_traj"].isin(n_4_test)]
    test = test.sort_values('time').reset_index(drop=True)

    train_data = get_data_in_format(train, dim_frame)
    test_data = get_data_in_format(test, dim_frame)


    return train_data, test_data

In [6]:
train, test = get_data_4_model(trajectories, 0.8, 1)

  0%|          | 0/4959859 [00:00<?, ?it/s]

  0%|          | 0/1260065 [00:00<?, ?it/s]

In [7]:
train.to_csv("shifted_2_1s.csv", mode="a", header=False, index=False)
test.to_csv("shifted_2_1s_test.csv", mode="a", header=False, index=False)


In [6]:
train.to_csv("completo_NOshifted_2_04s.csv", mode="a", header=False, index=False)
test.to_csv("completo_NOshifted_2_04s_test.csv", mode="a", header=False, index=False)
